# Hybrid Streaming Optimizer API Demo


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/imewei/NLSQ/blob/main/examples/notebooks/06_streaming/05_hybrid_streaming_api.ipynb)

This notebook demonstrates how to use `method='hybrid_streaming'` with both
`curve_fit()` and `curve_fit_large()` functions.

The hybrid streaming optimizer provides:
- Parameter normalization for better gradient signals
- L-BFGS warmup for robust initial convergence
- **4-layer defense strategy** for warmup divergence prevention (v0.3.6+)
- Streaming Gauss-Newton for exact covariance computation
- Automatic memory management for large datasets

In [1]:
# @title Install NLSQ (run once in Colab)
import sys

if 'google.colab' in sys.modules:
    print("Running in Google Colab - installing NLSQ...")
    !pip install -q nlsq
    print("NLSQ installed successfully!")
else:
    print("Not running in Colab - assuming NLSQ is already installed")

Not running in Colab - assuming NLSQ is already installed


In [2]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

In [3]:
import jax.numpy as jnp
import numpy as np

from nlsq import (
    HybridStreamingConfig,
    curve_fit,
    curve_fit_large,
    get_defense_telemetry,
    reset_defense_telemetry,
)

In [4]:
def exponential_decay(x, a, b, c):
    """Three-parameter exponential decay model."""
    return a * jnp.exp(-b * x) + c

In [5]:
# Generate synthetic data
np.random.seed(42)
x = np.linspace(0, 10, 2000)
true_params = np.array([5.0, 0.5, 1.0])
y_true = exponential_decay(x, *true_params)
y = y_true + np.random.normal(0, 0.1, len(x))

print(f"Dataset: {len(x)} samples")
print(f"True parameters: a={true_params[0]}, b={true_params[1]}, c={true_params[2]}")

Dataset: 2000 samples
True parameters: a=5.0, b=0.5, c=1.0


## Example 1: Basic usage with curve_fit()

In [6]:
result = curve_fit(
    exponential_decay,
    x,
    y,
    p0=np.array([4.0, 0.4, 0.8]),
    method='hybrid_streaming',
    verbose=1,
)

# Unpack result
popt, pcov = result

print(f"\nFitted parameters: {popt}")
print(f"True parameters:   {true_params}")
print(f"Parameter errors:  {np.abs(popt - true_params)}")
print(f"\nCovariance matrix diagonal: {np.diag(pcov)}")
print(f"Parameter std errors: {np.sqrt(np.diag(pcov))}")

Adaptive Hybrid Streaming Optimizer
Dataset size: 2,000 points
Parameters: 3
Normalization: auto
Precision: auto



Phase 0: Normalization setup complete (0.376s)
  Strategy: p0
  Precision: <class 'jax.numpy.float32'>

Phase 1: L-BFGS warmup...
  Precision: <class 'jax.numpy.float32'>


Phase 1 complete: 201 iterations (158.487s)
  Best loss: 9.754856e-03
  Switch reason: Gradient norm below threshold

Phase 2: Streaming Gauss-Newton...
  Precision: <class 'jax.numpy.float64'>
  Computing initial JTJ (1 chunks, 2,000 points)...


  Initial JTJ: 1/1 chunks (100%), elapsed=0.9s
  Initial JTJ complete: cost=1.950971e+01, time=0.9s


  GN iter 1/100: cost=1.950971e+01, grad_norm=1.084023e-13, reduction=0.000000e+00, Δ=0.5000, time=1.4s
Phase 2 complete: 1 iterations (2.360s)
  Final cost: 1.950971e+01
  Convergence: Gradient norm below tolerance
  Gradient norm: 1.084023e-13

Phase 3: Computing covariance...
  Precision: <class 'jax.numpy.float64'>


Phase 3 complete (0.558s)
  Residual variance (σ²): 9.769510e-03

Optimization Complete
Total time: 161.783s
Final parameters: [4.98595755 0.49962326 1.00657897]
Parameter std errors: [0.00988365 0.00213743 0.00433106]

Fitted parameters: [4.98595755 0.49962326 1.00657897]
True parameters:   [5.  0.5 1. ]
Parameter errors:  [0.01404245 0.00037674 0.00657897]

Covariance matrix diagonal: [9.76864863e-05 4.56862536e-06 1.87581078e-05]
Parameter std errors: [0.00988365 0.00213743 0.00433106]


## Example 2: With parameter bounds

In [7]:
result = curve_fit(
    exponential_decay,
    x,
    y,
    p0=np.array([4.0, 0.4, 0.8]),
    bounds=([0, 0, 0], [10, 2, 5]),
    method='hybrid_streaming',
    verbose=0,  # Silent mode
)

popt, pcov = result
print(f"Fitted parameters (bounded): {popt}")
print(f"Within bounds: {np.all(popt >= [0, 0, 0]) and np.all(popt <= [10, 2, 5])}")

  Computing initial JTJ (1 chunks, 2,000 points)...
  Initial JTJ: 1/1 chunks (100%), elapsed=0.1s
  Initial JTJ complete: cost=1.950971e+01, time=0.1s


  GN iter 1/100: cost=1.950971e+01, grad_norm=6.010757e-13, reduction=0.000000e+00, Δ=0.5000, time=0.2s
Fitted parameters (bounded): [4.98595755 0.49962326 1.00657897]
Within bounds: True


## Example 3: Large dataset with curve_fit_large()

In [8]:
# Generate larger dataset
x_large = np.linspace(0, 10, 10000)
y_large = exponential_decay(x_large, *true_params) + np.random.normal(0, 0.1, len(x_large))

popt, pcov = curve_fit_large(
    exponential_decay,
    x_large,
    y_large,
    p0=np.array([4.0, 0.4, 0.8]),
    method='hybrid_streaming',
    verbose=1,
)

print(f"\nFitted parameters (large dataset): {popt}")
print(f"True parameters:                   {true_params}")
print(f"Parameter errors:                  {np.abs(popt - true_params)}")

Adaptive Hybrid Streaming Optimizer
Dataset size: 10,000 points
Parameters: 3
Normalization: auto
Precision: auto

Phase 0: Normalization setup complete (0.001s)
  Strategy: p0
  Precision: <class 'jax.numpy.float32'>

Phase 1: L-BFGS warmup...
  Precision: <class 'jax.numpy.float32'>


Phase 1 complete: 201 iterations (151.546s)
  Best loss: 1.007906e-02
  Switch reason: Gradient norm below threshold

Phase 2: Streaming Gauss-Newton...
  Precision: <class 'jax.numpy.float64'>
  Computing initial JTJ (1 chunks, 10,000 points)...


  Initial JTJ: 1/1 chunks (100%), elapsed=0.5s
  Initial JTJ complete: cost=1.007906e+02, time=0.5s
  GN iter 1/100: cost=1.007906e+02, grad_norm=4.103577e-07, reduction=0.000000e+00, Δ=0.5000, time=0.2s
Phase 2 complete: 1 iterations (0.719s)
  Final cost: 1.007906e+02
  Convergence: Cost change below tolerance
  Gradient norm: 4.103577e-07

Phase 3: Computing covariance...
  Precision: <class 'jax.numpy.float64'>
Phase 3 complete (0.004s)
  Residual variance (σ²): 1.008208e-02

Optimization Complete
Total time: 152.272s
Final parameters: [5.00140479 0.50075551 0.99954281]
Parameter std errors: [0.00450281 0.0009709  0.00196412]

Fitted parameters (large dataset): [5.00140479 0.50075551 0.99954281]
True parameters:                   [5.  0.5 1. ]
Parameter errors:                  [0.00140479 0.00075551 0.00045719]


## Example 4: Custom configuration via kwargs

In [9]:
result = curve_fit(
    exponential_decay,
    x,
    y,
    p0=np.array([4.0, 0.4, 0.8]),
    method='hybrid_streaming',
    verbose=0,
    # HybridStreamingConfig overrides:
    warmup_iterations=300,
    normalization_strategy='p0',  # 'bounds' requires explicit bounds
    phase2_max_iterations=100,
)

popt, pcov = result
print(f"Fitted parameters (custom config): {popt}")
print(f"Result attributes available: {list(result.keys())[:10]}")

  Computing initial JTJ (1 chunks, 2,000 points)...
  Initial JTJ: 1/1 chunks (100%), elapsed=0.1s
  Initial JTJ complete: cost=1.950971e+01, time=0.1s


  GN iter 1/100: cost=1.950971e+01, grad_norm=1.084023e-13, reduction=0.000000e+00, Δ=0.5000, time=0.2s
Fitted parameters (custom config): [4.98595755 0.49962326 1.00657897]
Result attributes available: ['x', 'success', 'message', 'fun', 'pcov', 'perr', 'streaming_diagnostics', '_predictions_cache', '_residuals_cache', 'model']


## Example 5: Accessing full result details

In [10]:
result = curve_fit(
    exponential_decay,
    x,
    y,
    p0=np.array([4.0, 0.4, 0.8]),
    method='hybrid_streaming',
    verbose=0,
)

print(f"Success: {result['success']}")
print(f"Message: {result['message']}")
print(f"Final cost: {result.get('cost', 'N/A')}")

if 'streaming_diagnostics' in result:
    diag = result['streaming_diagnostics']
    print("\nStreaming diagnostics available:")
    print(f"  Keys: {list(diag.keys())}")

  Computing initial JTJ (1 chunks, 2,000 points)...
  Initial JTJ: 1/1 chunks (100%), elapsed=0.1s
  Initial JTJ complete: cost=1.950971e+01, time=0.1s


  GN iter 1/100: cost=1.950971e+01, grad_norm=1.084023e-13, reduction=0.000000e+00, Δ=0.5000, time=0.2s
Success: True
Message: Gradient norm below tolerance
Final cost: N/A

Streaming diagnostics available:
  Keys: ['phase_timings', 'phase_iterations', 'total_time', 'warmup_diagnostics', 'gauss_newton_diagnostics', 'phase_history']


---

## Example 6: 4-Layer Defense Strategy (v0.3.6+)

The hybrid streaming optimizer includes a **4-layer defense strategy** that prevents L-BFGS warmup divergence when initial parameters are already near optimal.

### The 4 Defense Layers:
1. **Layer 1: Warm Start Detection** - Skips warmup if initial loss < 1% of data variance
2. **Layer 2: Adaptive Step Size** - Scales step size based on initial fit quality
3. **Layer 3: Cost-Increase Guard** - Aborts if loss increases > 5% from initial
4. **Layer 4: Step Clipping** - Limits parameter update magnitude (max norm 0.1)

In [11]:
# Demonstrate defense layers with near-optimal starting point
reset_defense_telemetry()

# Near-optimal initial guess (simulating refinement scenario)
near_optimal_p0 = true_params * np.array([1.01, 0.99, 1.005])
print(f"Near-optimal p0: {near_optimal_p0}")
print(f"True params:     {true_params}")

# Defense layers are enabled by default
popt, pcov = curve_fit(
    exponential_decay,
    x, y,
    p0=near_optimal_p0,
    method='hybrid_streaming',
    verbose=0,
)

# Check telemetry
telemetry = get_defense_telemetry()
summary = telemetry.get_summary()
rates = telemetry.get_trigger_rates()

print(f"\nFitted params: {popt}")
print("\n--- Defense Layer Telemetry ---")
print(f"Layer 1 (warm start) rate: {rates.get('layer1_warm_start_rate', 0):.1f}%")
print(f"Layer 2 LR modes: {summary.get("layer2_lr_mode_counts", {})}")
print(f"Layer 3 (cost guard) triggers: {summary.get("layer3_cost_guard_triggers", 0)}")
print(f"Layer 4 (step clip) triggers: {summary.get("layer4_clip_triggers", 0)}")

INFO:nlsq.adaptive_hybrid_streaming:Phase 1: Skipping L-BFGS warmup - warm start detected (relative_loss=6.9218e-03)


Near-optimal p0: [5.05  0.495 1.005]
True params:     [5.  0.5 1. ]
  Computing initial JTJ (1 chunks, 2,000 points)...


  Initial JTJ: 1/1 chunks (100%), elapsed=0.1s
  Initial JTJ complete: cost=2.102160e+01, time=0.1s


  GN iter 1/100: cost=1.950980e+01, grad_norm=1.088125e+02, reduction=1.511805e+00, Δ=1.0000, time=0.5s


  GN iter 2/100: cost=1.950971e+01, grad_norm=7.088592e-01, reduction=8.461715e-05, Δ=1.0000, time=0.3s


  GN iter 3/100: cost=1.950971e+01, grad_norm=9.030406e-05, reduction=4.352074e-12, Δ=1.0000, time=0.3s

Fitted params: [4.98595755 0.49962326 1.00657897]

--- Defense Layer Telemetry ---
Layer 1 (warm start) rate: 100.0%
Layer 2 LR modes: {}
Layer 3 (cost guard) triggers: 0
Layer 4 (step clip) triggers: 0


### Example 7: Using Defense Layer Presets

In [12]:
# Available preset configurations
presets = {
    "Default": HybridStreamingConfig(),
    "Strict (refinement)": HybridStreamingConfig.defense_strict(),
    "Relaxed (exploration)": HybridStreamingConfig.defense_relaxed(),
    "Disabled (pre-0.3.6)": HybridStreamingConfig.defense_disabled(),
    "Scientific": HybridStreamingConfig.scientific_default(),
}

print("Defense Layer Presets:")
print(f"{'Preset':<25} {'L1':<5} {'L2':<5} {'L3':<5} {'L4':<5}")
print("-" * 50)

for name, config in presets.items():
    print(f"{name:<25} {'ON' if config.enable_warm_start_detection else 'OFF':<5} "
          f"{'ON' if config.enable_adaptive_warmup_lr else 'OFF':<5} "
          f"{'ON' if config.enable_cost_guard else 'OFF':<5} "
          f"{'ON' if config.enable_step_clipping else 'OFF':<5}")

Defense Layer Presets:
Preset                    L1    L2    L3    L4   
--------------------------------------------------
Default                   ON    ON    ON    ON   
Strict (refinement)       ON    ON    ON    ON   
Relaxed (exploration)     ON    ON    ON    ON   
Disabled (pre-0.3.6)      OFF   OFF   OFF   OFF  
Scientific                ON    ON    ON    ON   


In [13]:
# Use a preset for warm start refinement
config = HybridStreamingConfig.defense_strict()

popt, pcov = curve_fit(
    exponential_decay,
    x, y,
    p0=near_optimal_p0,
    method='hybrid_streaming',
    config=config,  # Pass preset config
    verbose=0,
)

print(f"Fitted with defense_strict(): {popt}")

INFO:nlsq.adaptive_hybrid_streaming:Phase 1: Skipping L-BFGS warmup - warm start detected (relative_loss=6.9218e-03)


  Computing initial JTJ (1 chunks, 2,000 points)...


  Initial JTJ: 1/1 chunks (100%), elapsed=0.2s
  Initial JTJ complete: cost=2.102160e+01, time=0.2s


  GN iter 1/100: cost=1.950980e+01, grad_norm=1.088125e+02, reduction=1.511805e+00, Δ=1.0000, time=0.3s


  GN iter 2/100: cost=1.950971e+01, grad_norm=7.088592e-01, reduction=8.461715e-05, Δ=1.0000, time=0.3s


  GN iter 3/100: cost=1.950971e+01, grad_norm=9.030406e-05, reduction=4.352074e-12, Δ=1.0000, time=0.3s
Fitted with defense_strict(): [4.98595755 0.49962326 1.00657897]


### Example 8: Custom Defense Configuration

In [14]:
# Fine-tune defense layers for specific needs
custom_config = HybridStreamingConfig(
    # Layer 1: Stricter warm start threshold
    enable_warm_start_detection=True,
    warm_start_threshold=0.005,  # 0.5% instead of 1%

    # Layer 2: Conservative learning rates
    enable_adaptive_warmup_lr=True,
    warmup_lr_refinement=1e-7,
    warmup_lr_careful=1e-6,

    # Layer 3: Tighter cost tolerance
    enable_cost_guard=True,
    cost_increase_tolerance=0.02,  # 2% instead of 5%

    # Layer 4: Smaller step limit
    enable_step_clipping=True,
    max_warmup_step_size=0.05,
)

popt, pcov = curve_fit(
    exponential_decay,
    x, y,
    p0=np.array([4.0, 0.4, 0.8]),
    method='hybrid_streaming',
    config=custom_config,
    verbose=0,
)

print(f"Fitted with custom defense config: {popt}")

  Computing initial JTJ (1 chunks, 2,000 points)...
  Initial JTJ: 1/1 chunks (100%), elapsed=0.1s
  Initial JTJ complete: cost=1.950971e+01, time=0.1s


  GN iter 1/100: cost=1.950971e+01, grad_norm=1.084023e-13, reduction=0.000000e+00, Δ=0.5000, time=0.2s
Fitted with custom defense config: [4.98595755 0.49962326 1.00657897]


### Example 9: Production Monitoring with Telemetry

In [15]:
# Production monitoring example
reset_defense_telemetry()

# Simulate batch of fits with varying quality
for i in range(3):
    noise = 0.01 if i < 3 else (0.3 if i < 7 else 1.0)
    p0 = true_params * (1 + np.random.uniform(-noise, noise, 3))

    curve_fit(
        exponential_decay, x, y, p0=p0,
        method='hybrid_streaming',
        verbose=0,
    )

# Get monitoring report
telemetry = get_defense_telemetry()
rates = telemetry.get_trigger_rates()

print("--- Production Monitoring Report (10 fits) ---")
print(f"Layer 1 (warm start):     {rates.get('layer1_warm_start_rate', 0):.1f}%")
print(f"Layer 2 (refinement LR):  {rates.get('layer2_refinement_rate', 0):.1f}%")
print(f"Layer 2 (careful LR):     {rates.get('layer2_careful_rate', 0):.1f}%")
print(f"Layer 2 (exploration LR): {rates.get('layer2_exploration_rate', 0):.1f}%")
print(f"Layer 3 (cost guard):     {rates.get('layer3_cost_guard_rate', 0):.1f}%")
print(f"Layer 4 (step clipping):  {rates.get('layer4_clip_rate', 0):.1f}%")

# Export Prometheus-compatible metrics
print("\n--- Prometheus Metrics ---")
for name, value in telemetry.export_metrics().items():
    print(f"{name}: {value}")

INFO:nlsq.adaptive_hybrid_streaming:Phase 1: Skipping L-BFGS warmup - warm start detected (relative_loss=6.4868e-03)


  Computing initial JTJ (1 chunks, 2,000 points)...


  Initial JTJ: 1/1 chunks (100%), elapsed=0.1s
  Initial JTJ complete: cost=1.970033e+01, time=0.1s


  GN iter 1/100: cost=1.950971e+01, grad_norm=3.390928e+01, reduction=1.906147e-01, Δ=1.0000, time=0.3s


INFO:nlsq.adaptive_hybrid_streaming:Phase 1: Skipping L-BFGS warmup - warm start detected (relative_loss=6.8572e-03)


...

  GN iter 3/100: cost=1.950971e+01, grad_norm=5.603807e-05, reduction=2.145839e-12, Δ=1.0000, time=0.4s
  Computing initial JTJ (1 chunks, 2,000 points)...


  Initial JTJ: 1/1 chunks (100%), elapsed=0.1s
  Initial JTJ complete: cost=2.040702e+01, time=0.1s


  GN iter 1/100: cost=1.950972e+01, grad_norm=7.189826e+01, reduction=8.973021e-01, Δ=1.0000, time=0.3s


  GN iter 2/100: cost=1.950971e+01, grad_norm=1.842586e-01, reduction=9.224318e-06, Δ=1.0000, time=0.3s


  GN iter 3/100: cost=1.950971e+01, grad_norm=4.903750e-06, reduction=4.973799e-14, Δ=1.0000, time=0.3s
--- Production Monitoring Report (10 fits) ---
Layer 1 (warm start):     100.0%
Layer 2 (refinement LR):  0.0%
Layer 2 (careful LR):     0.0%
Layer 2 (exploration LR): 0.0%
Layer 3 (cost guard):     0.0%
Layer 4 (step clipping):  0.0%

--- Prometheus Metrics ---
nlsq_defense_warmup_calls_total: 3
nlsq_defense_layer1_triggers_total: 3
nlsq_defense_layer2_refinement_total: 0
nlsq_defense_layer2_careful_total: 0
nlsq_defense_layer2_exploration_total: 0
nlsq_defense_layer3_triggers_total: 0
nlsq_defense_layer4_triggers_total: 0
nlsq_defense_lbfgs_history_fill_total: 0
nlsq_defense_lbfgs_line_search_failures_total: 0


---

## Summary

The hybrid streaming optimizer combines:
- **Parameter normalization** for better gradient signals when parameters have different scales
- **L-BFGS warmup** for robust initial convergence from poor initial guesses
- **4-layer defense strategy** (v0.3.6+) preventing warmup divergence near optimal
- **Streaming Gauss-Newton** for fast second-order convergence near the optimum
- **Exact covariance computation** for reliable uncertainty estimates

### Use `method='hybrid_streaming'` when:
- Parameters span many orders of magnitude
- Large datasets (100K+ points) with memory constraints
- Need production-quality uncertainty estimates
- Standard optimizers converge slowly
- Refining parameters from previous fits (warm starts)

### Defense Layer Presets:
| Scenario | Preset |
|----------|--------|
| Default usage | `HybridStreamingConfig()` |
| Warm start refinement | `HybridStreamingConfig.defense_strict()` |
| Exploration | `HybridStreamingConfig.defense_relaxed()` |
| Regression testing | `HybridStreamingConfig.defense_disabled()` |
| Scientific computing | `HybridStreamingConfig.scientific_default()` |

### Next Steps:
- [Defense Layers Demo](../05_feature_demos/defense_layers_demo.ipynb) - Deep dive into defense layers
- [Troubleshooting Guide](../03_advanced/troubleshooting_guide.ipynb) - Common issues and solutions